In [1]:
#Unzipping dataset
!unzip /content/layoffs.zip

Archive:  /content/layoffs.zip
  inflating: layoffs_data(3).csv     


In [2]:
#Importing the libraries required for now
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("layoffs_data(3).csv")

In [6]:
to_be_removed = data[(data["List_of_Employees_Laid_Off"]=="Unknown") & (data["Laid_Off_Count"].isna())]

In [7]:
to_be_removed.shape

(1141, 12)

In [8]:
removed = pd.merge(data, to_be_removed, how='left', indicator=True)
proper = removed[removed['_merge'] == 'left_only']

In [9]:
proper

,Company,Location_HQ,Industry,Percentage,Date,Source,Funds_Raised,Stage,Date_Added,Country,Laid_Off_Count,List_of_Employees_Laid_Off,_merge
1,Electronic Arts,SF Bay Area,Consumer,0.05,2024-02-28,https://www.cnbc.com/2024/02/28/ea-layoffs-com...,2.0,Post-IPO,2024-02-29 00:45:00,United States,670.0,Unknown,left_only
2,Vacasa,Portland,Travel,0.05,2024-02-28,https://www.oregonlive.com/business/2024/02/po...,834.0,Post-IPO,2024-02-29 17:22:37,United States,320.0,Unknown,left_only
3,Treasury Prime,SF Bay Area,Finance,0.40,2024-02-28,https://www.bankingdive.com/news/treasury-prim...,71.0,Series C,2024-02-29 00:48:03,United States,40.0,Unknown,left_only
5,Sony Interactive,SF Bay Area,Consumer,0.08,2024-02-27,https://www.theverge.com/2024/2/27/24084494/so...,NaN,Subsidiary,2024-02-27 18:49:42,United States,900.0,Unknown,left_only
6,Bumble,Austin,Consumer,0.30,2024-02-27,https://www.cnbc.com/2024/02/27/bumble-layoffs...,313.0,Post-IPO,2024-02-27 23:56:12,United States,350.0,Unknown,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3479,Help.com,Austin,Support,1.00,2020-03-16,LinkedIn,6.0,Seed,2020-04-16 06:36:06,United States,16.0,Unknown,left_only
3481,HopSkipDrive,Los Angeles,Transportation,0.10,2020-03-13,https://layoffs.fyi/2020/04/02/hopskipdrive-la...,45.0,Unknown,2020-04-03 04:47:03,United States,8.0,Unknown,left_only
3482,Panda Squad,SF Bay Area,Consumer,0.75,2020-03-13,https://twitter.com/danielsinger/status/123854...,1.0,Seed,2020-04-17 00:12:49,United States,6.0,Unknown,left_only
3483,Tamara Mellon,Los Angeles,Retail,0.40,2020-03-12,https://layoffs.fyi/list/tamara-mellon/,90.0,Series C,2020-03-31 16:04:23,United States,20.0,Unknown,left_only


In [10]:
proper[(proper["List_of_Employees_Laid_Off"]=="Unknown") & (proper["Laid_Off_Count"].isna())]

,Company,Location_HQ,Industry,Percentage,Date,Source,Funds_Raised,Stage,Date_Added,Country,Laid_Off_Count,List_of_Employees_Laid_Off,_merge


In [11]:
find = proper[(proper["List_of_Employees_Laid_Off"]!="Unknown") & (proper["Laid_Off_Count"].isna())]

In [12]:
find.shape

(54, 13)

In [13]:
pd.set_option('display.max_colwidth', None)

In [14]:
import re
find_google = find[find["List_of_Employees_Laid_Off"].str.contains(r"docs.google", regex=True)]

In [15]:
find_google.shape

(47, 13)

In [16]:
!pip install gspread oauth2client

In [17]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint as pp

scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]

In [18]:
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/apiforlayoffs-205380341afe.json",scope)
client = gspread.authorize(creds)

In [19]:
find_google['Index'] = range(len(find_google))

<ipython-input-19-fefa563a39e9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  find_google['Index'] = range(len(find_google))


In [20]:
def count_rows_and_update_dataframe(find_google, client, start_index=None, end_index=None):
    for index, row in find_google.iterrows():
        if row['Index'] is not None and row['Index'] < start_index:
            continue  # Skip rows until the start_index is reached

        if row['Index'] is not None and row['Index'] > end_index:
            break  # Exit the loop if the current index exceeds the specified end_index

        column_value = row['List_of_Employees_Laid_Off']
        try:
            sheet = client.open_by_url(column_value)
            total_rows = 0  # Initialize total rows counter

            for worksheet in sheet.worksheets():
                # Count the number of rows in the current worksheet
                num_rows = len(worksheet.get_all_values())
                total_rows += num_rows  # Add the number of rows to the total

            # Save the total number of rows to the "Laid_Off_Count" column
            find_google.at[index, "Laid_Off_Count"] = total_rows
        except Exception as e:
            print(f"Error accessing Google Sheets document: {e}")
            continue

In [21]:
import time

# Define the range with intervals of 5 until index 47
for i in range(0, 48, 5):
    count_rows_and_update_dataframe(find_google, client, start_index=i, end_index=min(i + 4, 47))
    time.sleep(30)  # Add a delay of 1 second between iterations


Error accessing Google Sheets document: 
Error accessing Google Sheets document: {'code': 404, 'message': 'Requested entity was not found.', 'status': 'NOT_FOUND'}
Error accessing Google Sheets document: {'code': 404, 'message': 'Requested entity was not found.', 'status': 'NOT_FOUND'}
Error accessing Google Sheets document: {'code': 403, 'message': 'The caller does not have permission', 'status': 'PERMISSION_DENIED'}
Error accessing Google Sheets document: 
Error accessing Google Sheets document: 
Error accessing Google Sheets document: {'code': 403, 'message': 'The caller does not have permission', 'status': 'PERMISSION_DENIED'}
Error accessing Google Sheets document: {'code': 404, 'message': 'Requested entity was not found.', 'status': 'NOT_FOUND'}
Error accessing Google Sheets document: {'code': 404, 'message': 'Requested entity was not found.', 'status': 'NOT_FOUND'}


In [22]:
#Need to remove Index Col
#Merge find_google with find
#Try to find layoff count for other websites
find_google.drop(columns=['Index'], inplace=True)

<ipython-input-22-c90a68f8ac99>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  find_google.drop(columns=['Index'], inplace=True)


In [23]:
find_google

,Company,Location_HQ,Industry,Percentage,Date,Source,Funds_Raised,Stage,Date_Added,Country,Laid_Off_Count,List_of_Employees_Laid_Off,_merge
94,Flexport,SF Bay Area,Logistics,0.20,2024-01-26,https://www.theinformation.com/articles/flexport-to-lay-off-nearly-20-of-staff-in-latest-cuts,2400.0,Series E,2024-01-26 19:48:39,United States,130.0,https://docs.google.com/spreadsheets/d/1d6ejecq8qy46SJv_VJk2TMbzT_S_77EGHPHjYMcJruw/edit?resourcekey#gid=550780554,left_only
108,HubSpot,Boston,Marketing,NaN,2024-01-24,https://www.linkedin.com/posts/jcolman_hubspot-content-designers-open-to-work-activity-7156050968306991105-917M/,100.0,Post-IPO,2024-01-25 14:35:26,United States,46.0,https://docs.google.com/spreadsheets/d/1AzNRk1EflTBUtFI1MLqle9br27Oc0U5s7ioOhTpeKbM/edit#gid=0,left_only
140,Vendr,Boston,Other,NaN,2024-01-12,https://www.vendr.com/blog/vendr-organizational-update,216.0,Series B,2024-01-13 16:10:09,United States,NaN,https://docs.google.com/forms/d/1WaKbqSwI-we9qYb3VysOTGPvoR3qjLu35GYk4KbQa3E/viewform?edit_requested=true,left_only
216,Curalie,Berlin,Healthcare,1.00,2023-12-14,https://www.msn.com/de-de/finanzen/top-stories/fresenius-aktie-gibt-nach-fresenius-stampft-digitaltochter-curlie-ein/ar-AA1lxOnW,NaN,Unknown,2024-01-02 03:47:04,Germany,51.0,https://docs.google.com/spreadsheets/d/1n8cNFFPmOwSkny5ruqfYA4tTwonhwnYWgvX7zpA-p6Y/edit#gid=0,left_only
244,Course Hero,SF Bay Area,Education,0.25,2023-12-05,https://www.linkedin.com/posts/coursehero_course-hero-alumni-talent-activity-7137848787347132416-q9KI/,477.0,Series C,2023-12-08 17:50:24,United States,21.0,https://docs.google.com/spreadsheets/d/1CLF3m4H8m8l-ueiRqus7FTSZyOyPkcfq-1jWNARolbc/edit#gid=1497948505,left_only
307,Nextdoor,SF Bay Area,Consumer,0.25,2023-11-07,https://investors.nextdoor.com/news/news-details/2023/Nextdoor-Reports-Third-Quarter-2023-Results/,736.0,Post-IPO,2023-11-07 22:22:22,United States,NaN,https://docs.google.com/spreadsheets/d/e/2PACX-1vQb4VFkL-FC5XiNI45fBWzYLnP4cyBvOC0KUNG8GZV4nB9wc1_0RFuzPv8u75bQ6I_blsuiwtGLLBkm/pubhtml?gid=0&single=true,left_only
366,Flexport,SF Bay Area,Logistics,0.20,2023-10-13,https://www.cnbc.com/2023/10/12/flexport-is-laying-off-20percent-of-its-workforce.html,2400.0,Series E,2023-10-12 22:18:24,United States,286.0,https://docs.google.com/spreadsheets/d/1jPHAa9OpCvSnWEDXRia7_MfxJs0duew4laVFNdOUt8c/edit#gid=1186596436,left_only
367,Uno Health,New York City,Healthcare,NaN,2023-10-13,https://docs.google.com/spreadsheets/d/1uCA0BFMYjCFYeoC7ilv5FvA-oCPGHLPKJZ8xjql_K54/edit#gid=0,NaN,Unknown,2023-10-17 15:49:26,United States,16.0,https://docs.google.com/spreadsheets/d/1uCA0BFMYjCFYeoC7ilv5FvA-oCPGHLPKJZ8xjql_K54/edit#gid=0,left_only
404,Sono Motors,Munich,Transportation,NaN,2023-10-02,https://www.linkedin.com/feed/update/urn:li:activity:7114544303233740800/,127.0,Post-IPO,2023-10-02 15:50:34,Germany,25.0,https://docs.google.com/spreadsheets/d/1ueOI0brbtSv4tQ4G49bttz5iv2JbnkOxait0unFJ0wM/edit#gid=0,left_only
500,JOIN,Berlin,Recruiting,NaN,2023-08-23,https://docs.google.com/spreadsheets/d/1bsu6O2DCEkzM-4Ny8q6HmLZ2DGfjpZVYdxTOgg3z23M/edit?resourcekey#gid=2104038854,NaN,Unknown,2023-08-25 20:49:41,Germany,5.0,https://docs.google.com/spreadsheets/d/1bsu6O2DCEkzM-4Ny8q6HmLZ2DGfjpZVYdxTOgg3z23M/edit?resourcekey#gid=2104038854,left_only


In [24]:
find_google.shape

(47, 13)

In [25]:
not_found = find_google[(find_google["List_of_Employees_Laid_Off"]!="Unknown") & (find_google["Laid_Off_Count"].isna())]

In [26]:
not_found.shape

(9, 13)

In [32]:
find_others = find[~find["List_of_Employees_Laid_Off"].str.contains(r"docs.google", regex=True)]

In [33]:
find_others.shape

(7, 13)

In [34]:
find_others

,Company,Location_HQ,Industry,Percentage,Date,Source,Funds_Raised,Stage,Date_Added,Country,Laid_Off_Count,List_of_Employees_Laid_Off,_merge
1173,Reserve,SF Bay Area,Crypto,NaN,2023-02-17,https://www.linkedin.com/posts/constanza-ag%C3%BCero-ingravidi_job-board-reserves-talent-activity-7030942387006750720-abHD/,NaN,Unknown,2023-03-03 23:18:13,United States,NaN,https://myreserve.notion.site/myreserve/Job-board-Reserve-s-talent-8a042fa4c8364012b4ebd9d1c4897e1c,left_only
1414,Mudafy,Mexico City,Real Estate,0.70,2023-01-19,https://www.cronista.com/infotechnology/actualidad/vuelven-los-despidos-masivos-la-empresa-mudafy-echo-al-90-de-sus-empleados/,13.0,Series A,2023-01-20 01:19:18,United States,NaN,https://mudafy.notion.site/mudafy/70169b5afaa1403ba2f3efd7d863fc72?v=b482d6e10f0b4f47a5e78fabf2fdb4a2,left_only
1438,Coda,SF Bay Area,Other,NaN,2023-01-18,Internal memo,240.0,Series D,2023-02-16 15:41:58,United States,NaN,https://coda.io/@kenny/coda-alumni-list,left_only
1814,Hopin,London,Other,0.17,2022-11-16,https://www.exhibitionworld.co.uk/hopin-announces-new-layoffs-and-a-transformation-with-new-product-launches,1000.0,Series D,2022-11-17 01:00:45,United Kingdom,NaN,https://hopin.softr.app/,left_only
1864,Avast,Phoenix,Security,0.25,2022-11-10,https://cijeurope.com/en/gen-digital-to-lay-off-about-a-quarter-of-its-staff-in-czechia/post.html,NaN,Unknown,2022-11-17 04:19:58,United States,NaN,https://www.weworkedatavastnorton.com/,left_only
1881,Amobee,SF Bay Area,Marketing,NaN,2022-11-09,https://www.linkedin.com/feed/update/urn:li:activity:6996113373260054528/,72.0,Acquired,2022-11-13 06:49:04,United States,NaN,https://www.amobee.com/alumni/,left_only
1971,Convoy,Seattle,Logistics,NaN,2022-10-25,https://www.geekwire.com/2022/convoy-lays-off-more-employees-as-part-of-reorganization-at-seattle-trucking-startup/,1100.0,Series E,2022-10-25 15:13:12,United States,NaN,https://coda.io/d/Talent-Board_dN7cqX2rCM4/Candidates_suM29,left_only


In [35]:
find_others.loc[1173,'Laid_Off_Count']=48
find_others.loc[1414,'Laid_Off_Count']=93
find_others.loc[1438,'Laid_Off_Count']=35
# find_others.loc[52554,'Laid_Off_Count'] - NOT AVAILABLE
# find_others.loc[53270,'Laid_Off_Count'] - NOT AVAILABLE
# find_others.loc[53591,'Laid_Off_Count'] - NOT AVAILABLE
find_others.loc[1971,'Laid_Off_Count']=97
# Assumption-too many inconsistency

In [36]:
find_others

,Company,Location_HQ,Industry,Percentage,Date,Source,Funds_Raised,Stage,Date_Added,Country,Laid_Off_Count,List_of_Employees_Laid_Off,_merge
1173,Reserve,SF Bay Area,Crypto,NaN,2023-02-17,https://www.linkedin.com/posts/constanza-ag%C3%BCero-ingravidi_job-board-reserves-talent-activity-7030942387006750720-abHD/,NaN,Unknown,2023-03-03 23:18:13,United States,48.0,https://myreserve.notion.site/myreserve/Job-board-Reserve-s-talent-8a042fa4c8364012b4ebd9d1c4897e1c,left_only
1414,Mudafy,Mexico City,Real Estate,0.70,2023-01-19,https://www.cronista.com/infotechnology/actualidad/vuelven-los-despidos-masivos-la-empresa-mudafy-echo-al-90-de-sus-empleados/,13.0,Series A,2023-01-20 01:19:18,United States,93.0,https://mudafy.notion.site/mudafy/70169b5afaa1403ba2f3efd7d863fc72?v=b482d6e10f0b4f47a5e78fabf2fdb4a2,left_only
1438,Coda,SF Bay Area,Other,NaN,2023-01-18,Internal memo,240.0,Series D,2023-02-16 15:41:58,United States,35.0,https://coda.io/@kenny/coda-alumni-list,left_only
1814,Hopin,London,Other,0.17,2022-11-16,https://www.exhibitionworld.co.uk/hopin-announces-new-layoffs-and-a-transformation-with-new-product-launches,1000.0,Series D,2022-11-17 01:00:45,United Kingdom,NaN,https://hopin.softr.app/,left_only
1864,Avast,Phoenix,Security,0.25,2022-11-10,https://cijeurope.com/en/gen-digital-to-lay-off-about-a-quarter-of-its-staff-in-czechia/post.html,NaN,Unknown,2022-11-17 04:19:58,United States,NaN,https://www.weworkedatavastnorton.com/,left_only
1881,Amobee,SF Bay Area,Marketing,NaN,2022-11-09,https://www.linkedin.com/feed/update/urn:li:activity:6996113373260054528/,72.0,Acquired,2022-11-13 06:49:04,United States,NaN,https://www.amobee.com/alumni/,left_only
1971,Convoy,Seattle,Logistics,NaN,2022-10-25,https://www.geekwire.com/2022/convoy-lays-off-more-employees-as-part-of-reorganization-at-seattle-trucking-startup/,1100.0,Series E,2022-10-25 15:13:12,United States,97.0,https://coda.io/d/Talent-Board_dN7cqX2rCM4/Candidates_suM29,left_only


In [37]:
proper_subset = find.fillna(find_google)

In [38]:
proper_subset.shape

(54, 13)

In [39]:
proper_subset = proper_subset.fillna(find_others)

In [40]:
proper_subset

,Company,Location_HQ,Industry,Percentage,Date,Source,Funds_Raised,Stage,Date_Added,Country,Laid_Off_Count,List_of_Employees_Laid_Off,_merge
94,Flexport,SF Bay Area,Logistics,0.20,2024-01-26,https://www.theinformation.com/articles/flexport-to-lay-off-nearly-20-of-staff-in-latest-cuts,2400.0,Series E,2024-01-26 19:48:39,United States,130.0,https://docs.google.com/spreadsheets/d/1d6ejecq8qy46SJv_VJk2TMbzT_S_77EGHPHjYMcJruw/edit?resourcekey#gid=550780554,left_only
108,HubSpot,Boston,Marketing,NaN,2024-01-24,https://www.linkedin.com/posts/jcolman_hubspot-content-designers-open-to-work-activity-7156050968306991105-917M/,100.0,Post-IPO,2024-01-25 14:35:26,United States,46.0,https://docs.google.com/spreadsheets/d/1AzNRk1EflTBUtFI1MLqle9br27Oc0U5s7ioOhTpeKbM/edit#gid=0,left_only
140,Vendr,Boston,Other,NaN,2024-01-12,https://www.vendr.com/blog/vendr-organizational-update,216.0,Series B,2024-01-13 16:10:09,United States,NaN,https://docs.google.com/forms/d/1WaKbqSwI-we9qYb3VysOTGPvoR3qjLu35GYk4KbQa3E/viewform?edit_requested=true,left_only
216,Curalie,Berlin,Healthcare,1.00,2023-12-14,https://www.msn.com/de-de/finanzen/top-stories/fresenius-aktie-gibt-nach-fresenius-stampft-digitaltochter-curlie-ein/ar-AA1lxOnW,NaN,Unknown,2024-01-02 03:47:04,Germany,51.0,https://docs.google.com/spreadsheets/d/1n8cNFFPmOwSkny5ruqfYA4tTwonhwnYWgvX7zpA-p6Y/edit#gid=0,left_only
244,Course Hero,SF Bay Area,Education,0.25,2023-12-05,https://www.linkedin.com/posts/coursehero_course-hero-alumni-talent-activity-7137848787347132416-q9KI/,477.0,Series C,2023-12-08 17:50:24,United States,21.0,https://docs.google.com/spreadsheets/d/1CLF3m4H8m8l-ueiRqus7FTSZyOyPkcfq-1jWNARolbc/edit#gid=1497948505,left_only
307,Nextdoor,SF Bay Area,Consumer,0.25,2023-11-07,https://investors.nextdoor.com/news/news-details/2023/Nextdoor-Reports-Third-Quarter-2023-Results/,736.0,Post-IPO,2023-11-07 22:22:22,United States,NaN,https://docs.google.com/spreadsheets/d/e/2PACX-1vQb4VFkL-FC5XiNI45fBWzYLnP4cyBvOC0KUNG8GZV4nB9wc1_0RFuzPv8u75bQ6I_blsuiwtGLLBkm/pubhtml?gid=0&single=true,left_only
366,Flexport,SF Bay Area,Logistics,0.20,2023-10-13,https://www.cnbc.com/2023/10/12/flexport-is-laying-off-20percent-of-its-workforce.html,2400.0,Series E,2023-10-12 22:18:24,United States,286.0,https://docs.google.com/spreadsheets/d/1jPHAa9OpCvSnWEDXRia7_MfxJs0duew4laVFNdOUt8c/edit#gid=1186596436,left_only
367,Uno Health,New York City,Healthcare,NaN,2023-10-13,https://docs.google.com/spreadsheets/d/1uCA0BFMYjCFYeoC7ilv5FvA-oCPGHLPKJZ8xjql_K54/edit#gid=0,NaN,Unknown,2023-10-17 15:49:26,United States,16.0,https://docs.google.com/spreadsheets/d/1uCA0BFMYjCFYeoC7ilv5FvA-oCPGHLPKJZ8xjql_K54/edit#gid=0,left_only
404,Sono Motors,Munich,Transportation,NaN,2023-10-02,https://www.linkedin.com/feed/update/urn:li:activity:7114544303233740800/,127.0,Post-IPO,2023-10-02 15:50:34,Germany,25.0,https://docs.google.com/spreadsheets/d/1ueOI0brbtSv4tQ4G49bttz5iv2JbnkOxait0unFJ0wM/edit#gid=0,left_only
500,JOIN,Berlin,Recruiting,NaN,2023-08-23,https://docs.google.com/spreadsheets/d/1bsu6O2DCEkzM-4Ny8q6HmLZ2DGfjpZVYdxTOgg3z23M/edit?resourcekey#gid=2104038854,NaN,Unknown,2023-08-25 20:49:41,Germany,5.0,https://docs.google.com/spreadsheets/d/1bsu6O2DCEkzM-4Ny8q6HmLZ2DGfjpZVYdxTOgg3z23M/edit?resourcekey#gid=2104038854,left_only


In [41]:
tp = proper_subset[(proper_subset["List_of_Employees_Laid_Off"]!="Unknown") & (proper_subset["Laid_Off_Count"].isna())]

In [42]:
tp

,Company,Location_HQ,Industry,Percentage,Date,Source,Funds_Raised,Stage,Date_Added,Country,Laid_Off_Count,List_of_Employees_Laid_Off,_merge
140,Vendr,Boston,Other,NaN,2024-01-12,https://www.vendr.com/blog/vendr-organizational-update,216.0,Series B,2024-01-13 16:10:09,United States,NaN,https://docs.google.com/forms/d/1WaKbqSwI-we9qYb3VysOTGPvoR3qjLu35GYk4KbQa3E/viewform?edit_requested=true,left_only
307,Nextdoor,SF Bay Area,Consumer,0.25,2023-11-07,https://investors.nextdoor.com/news/news-details/2023/Nextdoor-Reports-Third-Quarter-2023-Results/,736.0,Post-IPO,2023-11-07 22:22:22,United States,NaN,https://docs.google.com/spreadsheets/d/e/2PACX-1vQb4VFkL-FC5XiNI45fBWzYLnP4cyBvOC0KUNG8GZV4nB9wc1_0RFuzPv8u75bQ6I_blsuiwtGLLBkm/pubhtml?gid=0&single=true,left_only
538,StyleSeat,SF Bay Area,Consumer,0.17,2023-08-09,https://docs.google.com/spreadsheets/d/1q4eOXAZiboWrj3goaBeOKfGOKP3ONsVQkXxDH1nXZw0/edit#gid=0,40.0,Series C,2023-10-02 16:08:11,United States,NaN,https://docs.google.com/spreadsheets/d/1q4eOXAZiboWrj3goaBeOKfGOKP3ONsVQkXxDH1nXZw0/edit#gid=0,left_only
1324,Oyster,Charlotte,HR,NaN,2023-01-30,https://www.linkedin.com/posts/teljamou_oyster-talent-list-activity-7026112666393829376-KB_N/,224.0,Series C,2023-02-01 05:15:18,United States,NaN,https://docs.google.com/spreadsheets/d/1RYg-4AkvXU-plkpY9_4D89f17i_tXt8S3Yfy3E0I3U0/edit,left_only
1434,Aiven,Helsinki,Infrastructure,0.20,2023-01-18,https://aiven.io/blog/a-message-from-our-ceo-oskari-saarenmaa,420.0,Series D,2023-01-18 16:01:39,Finland,NaN,https://docs.google.com/spreadsheets/u/3/d/e/2PACX-1vSZBSGZPopyiSBwH9jQd7pn3UzPjDQrGPFLmeoNEWNd89zr17PBUM_QPRPafoxnp9ARLuOdcZg_SLLM/pubhtml,left_only
1711,Strava,SF Bay Area,Fitness,0.14,2022-12-01,https://www.bicycleretailer.com/industry-news/2022/12/05/layoffs-affect-about-15-staff-strava#.Y5Jcu-zMK8A,151.0,Series F,2022-12-04 22:56:18,United States,NaN,https://docs.google.com/spreadsheets/u/1/d/e/2PACX-1vSm6vCc5Qlyt_RGpugfuvSFlnMjj0C2jU6JwZhc2CNhEuet88bbcDwmRu07eocW0QI9YITWPhNcQ7UO/pubhtml,left_only
1738,MessageBird,Amsterdam,Other,0.31,2022-11-29,https://tech.eu/2022/11/29/messagebird-lay-offs-close-to-a-third-of-the/,1100.0,Series C,2022-11-30 05:57:28,Netherlands,NaN,https://docs.google.com/spreadsheets/d/1n7SH4I4E16-Y7opqnKxHw1WHwIOIQIJg_BaEzrAG3F0/edit#gid=1288855899,left_only
1814,Hopin,London,Other,0.17,2022-11-16,https://www.exhibitionworld.co.uk/hopin-announces-new-layoffs-and-a-transformation-with-new-product-launches,1000.0,Series D,2022-11-17 01:00:45,United Kingdom,NaN,https://hopin.softr.app/,left_only
1829,Nirvana Money,Miami,Finance,1.00,2022-11-15,https://twitter.com/_Liso_/status/1592561040898019328,NaN,Unknown,2022-11-16 00:08:40,United States,NaN,https://docs.google.com/spreadsheets/d/1lfYAVNzpQaYteMqtKGzqHqHxRF0E2lNMrQGj_bfBR4M/edit#gid=0,left_only
1864,Avast,Phoenix,Security,0.25,2022-11-10,https://cijeurope.com/en/gen-digital-to-lay-off-about-a-quarter-of-its-staff-in-czechia/post.html,NaN,Unknown,2022-11-17 04:19:58,United States,NaN,https://www.weworkedatavastnorton.com/,left_only


In [43]:
tp.shape

(12, 13)

In [44]:
Final = proper.fillna(proper_subset)

In [45]:
Final.shape

(2344, 13)

In [47]:
Final.index

Int64Index([   1,    2,    3,    5,    6,    7,    9,   10,   12,   13,
            ...
            3471, 3472, 3473, 3477, 3478, 3479, 3481, 3482, 3483, 3484],
           dtype='int64', length=2344)

In [48]:
proper.index

Int64Index([   1,    2,    3,    5,    6,    7,    9,   10,   12,   13,
            ...
            3471, 3472, 3473, 3477, 3478, 3479, 3481, 3482, 3483, 3484],
           dtype='int64', length=2344)

In [49]:
Final[(Final["List_of_Employees_Laid_Off"]=="Unknown") & (Final["Laid_Off_Count"].isna())].shape

(0, 13)

In [50]:
Final[(Final["List_of_Employees_Laid_Off"]!="Unknown") & (Final["Laid_Off_Count"].isna())].shape

(12, 13)

In [51]:
Final.shape

(2344, 13)

In [52]:
condition =  (Final["List_of_Employees_Laid_Off"] != "Unknown") & (Final["Laid_Off_Count"].isna())

# Filtering out rows based on the condition
Final_dataset = Final[~condition]

In [53]:
Final_dataset[(Final_dataset["List_of_Employees_Laid_Off"]=="Unknown") & (Final_dataset["Laid_Off_Count"].isna())].shape

(0, 13)

In [54]:
Final_dataset[(Final_dataset["List_of_Employees_Laid_Off"]!="Unknown") & (Final_dataset["Laid_Off_Count"].isna())].shape

(0, 13)

In [57]:
Final_dataset.shape

(2332, 13)

In [59]:
Final_dataset

,Company,Location_HQ,Industry,Percentage,Date,Source,Funds_Raised,Stage,Date_Added,Country,Laid_Off_Count,List_of_Employees_Laid_Off,_merge
1,Electronic Arts,SF Bay Area,Consumer,0.05,2024-02-28,https://www.cnbc.com/2024/02/28/ea-layoffs-company-to-cut-5percent-of-workforce-or-about-670-employees.html,2.0,Post-IPO,2024-02-29 00:45:00,United States,670.0,Unknown,left_only
2,Vacasa,Portland,Travel,0.05,2024-02-28,https://www.oregonlive.com/business/2024/02/portland-based-vacasa-will-lay-off-another-320-new-year-is-off-to-a-difficult-start.html,834.0,Post-IPO,2024-02-29 17:22:37,United States,320.0,Unknown,left_only
3,Treasury Prime,SF Bay Area,Finance,0.40,2024-02-28,https://www.bankingdive.com/news/treasury-prime-lay-off-40-50-employees-marketing-liaison-direct-fintech-partner-chris-dean/708770/,71.0,Series C,2024-02-29 00:48:03,United States,40.0,Unknown,left_only
5,Sony Interactive,SF Bay Area,Consumer,0.08,2024-02-27,https://www.theverge.com/2024/2/27/24084494/sony-playstation-layoffs-2024,NaN,Subsidiary,2024-02-27 18:49:42,United States,900.0,Unknown,left_only
6,Bumble,Austin,Consumer,0.30,2024-02-27,https://www.cnbc.com/2024/02/27/bumble-layoffs-350-employees-or-about-37percent-of-workforce.html,313.0,Post-IPO,2024-02-27 23:56:12,United States,350.0,Unknown,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3479,Help.com,Austin,Support,1.00,2020-03-16,LinkedIn,6.0,Seed,2020-04-16 06:36:06,United States,16.0,Unknown,left_only
3481,HopSkipDrive,Los Angeles,Transportation,0.10,2020-03-13,https://layoffs.fyi/2020/04/02/hopskipdrive-laid-off-10-of-team-due-to-school-closures/,45.0,Unknown,2020-04-03 04:47:03,United States,8.0,Unknown,left_only
3482,Panda Squad,SF Bay Area,Consumer,0.75,2020-03-13,https://twitter.com/danielsinger/status/1238545571092160514,1.0,Seed,2020-04-17 00:12:49,United States,6.0,Unknown,left_only
3483,Tamara Mellon,Los Angeles,Retail,0.40,2020-03-12,https://layoffs.fyi/list/tamara-mellon/,90.0,Series C,2020-03-31 16:04:23,United States,20.0,Unknown,left_only


In [60]:
Final_dataset.drop('_merge', axis=1, inplace=True)

<ipython-input-60-b5f579853a66>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_dataset.drop('_merge', axis=1, inplace=True)


In [61]:
Final_dataset.shape

(2332, 12)

In [62]:
Final_dataset

,Company,Location_HQ,Industry,Percentage,Date,Source,Funds_Raised,Stage,Date_Added,Country,Laid_Off_Count,List_of_Employees_Laid_Off
1,Electronic Arts,SF Bay Area,Consumer,0.05,2024-02-28,https://www.cnbc.com/2024/02/28/ea-layoffs-company-to-cut-5percent-of-workforce-or-about-670-employees.html,2.0,Post-IPO,2024-02-29 00:45:00,United States,670.0,Unknown
2,Vacasa,Portland,Travel,0.05,2024-02-28,https://www.oregonlive.com/business/2024/02/portland-based-vacasa-will-lay-off-another-320-new-year-is-off-to-a-difficult-start.html,834.0,Post-IPO,2024-02-29 17:22:37,United States,320.0,Unknown
3,Treasury Prime,SF Bay Area,Finance,0.40,2024-02-28,https://www.bankingdive.com/news/treasury-prime-lay-off-40-50-employees-marketing-liaison-direct-fintech-partner-chris-dean/708770/,71.0,Series C,2024-02-29 00:48:03,United States,40.0,Unknown
5,Sony Interactive,SF Bay Area,Consumer,0.08,2024-02-27,https://www.theverge.com/2024/2/27/24084494/sony-playstation-layoffs-2024,NaN,Subsidiary,2024-02-27 18:49:42,United States,900.0,Unknown
6,Bumble,Austin,Consumer,0.30,2024-02-27,https://www.cnbc.com/2024/02/27/bumble-layoffs-350-employees-or-about-37percent-of-workforce.html,313.0,Post-IPO,2024-02-27 23:56:12,United States,350.0,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...
3479,Help.com,Austin,Support,1.00,2020-03-16,LinkedIn,6.0,Seed,2020-04-16 06:36:06,United States,16.0,Unknown
3481,HopSkipDrive,Los Angeles,Transportation,0.10,2020-03-13,https://layoffs.fyi/2020/04/02/hopskipdrive-laid-off-10-of-team-due-to-school-closures/,45.0,Unknown,2020-04-03 04:47:03,United States,8.0,Unknown
3482,Panda Squad,SF Bay Area,Consumer,0.75,2020-03-13,https://twitter.com/danielsinger/status/1238545571092160514,1.0,Seed,2020-04-17 00:12:49,United States,6.0,Unknown
3483,Tamara Mellon,Los Angeles,Retail,0.40,2020-03-12,https://layoffs.fyi/list/tamara-mellon/,90.0,Series C,2020-03-31 16:04:23,United States,20.0,Unknown


In [65]:
Final_dataset.to_csv("final_data.csv", index=False)

In [66]:
data = pd.read_csv("final_data.csv")

In [68]:
data.head()

,Company,Location_HQ,Industry,Percentage,Date,Source,Funds_Raised,Stage,Date_Added,Country,Laid_Off_Count,List_of_Employees_Laid_Off
0,Electronic Arts,SF Bay Area,Consumer,0.05,2024-02-28,https://www.cnbc.com/2024/02/28/ea-layoffs-company-to-cut-5percent-of-workforce-or-about-670-employees.html,2.0,Post-IPO,2024-02-29 00:45:00,United States,670.0,Unknown
1,Vacasa,Portland,Travel,0.05,2024-02-28,https://www.oregonlive.com/business/2024/02/portland-based-vacasa-will-lay-off-another-320-new-year-is-off-to-a-difficult-start.html,834.0,Post-IPO,2024-02-29 17:22:37,United States,320.0,Unknown
2,Treasury Prime,SF Bay Area,Finance,0.40,2024-02-28,https://www.bankingdive.com/news/treasury-prime-lay-off-40-50-employees-marketing-liaison-direct-fintech-partner-chris-dean/708770/,71.0,Series C,2024-02-29 00:48:03,United States,40.0,Unknown
3,Sony Interactive,SF Bay Area,Consumer,0.08,2024-02-27,https://www.theverge.com/2024/2/27/24084494/sony-playstation-layoffs-2024,NaN,Subsidiary,2024-02-27 18:49:42,United States,900.0,Unknown
4,Bumble,Austin,Consumer,0.30,2024-02-27,https://www.cnbc.com/2024/02/27/bumble-layoffs-350-employees-or-about-37percent-of-workforce.html,313.0,Post-IPO,2024-02-27 23:56:12,United States,350.0,Unknown


In [69]:
data[(data["List_of_Employees_Laid_Off"]=="Unknown") & (data["Laid_Off_Count"].isna())].shape

(0, 12)

In [70]:
data[(data["List_of_Employees_Laid_Off"]!="Unknown") & (data["Laid_Off_Count"].isna())].shape

(0, 12)